In [2]:
import os
import gc
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
# from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model
from keras.utils import Sequence

In [3]:
path = "CutomData[ISL]"
files=os.listdir(path)
files.sort()

image_arr=[]
label_arr=[]

#loop through each file
for i in range(len(files)):
    #list of images in each folder
    sub_file = os.listdir(path+"/"+files[i])
    #loop through each sub_file
    for j in range(len(sub_file)):
        file_path = path+"/"+files[i]+"/"+sub_file[j]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image_arr.append(image)
        label_arr.append(i)

#now converting list to numpy array
image_arr = np.array(image_arr)
label_arr = np.array(label_arr,dtype="float")
#splitting training and testing data
X_train,X_test,Y_train,Y_test = train_test_split(image_arr,label_arr,test_size=0.15)

del image_arr,label_arr

# to free memory 
gc.collect()

model=Sequential()
#add pretraind model to sequential model, we are using EfficientNetB0
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape=(128,128,3), include_top=False)
model.add(pretrained_model)

#adding Pooling layer
model.add(layers.GlobalAveragePooling2D())

#adding Dropout layer
#we create dropout to increase accuracy by reducing overfitting
model.add(layers.Dropout(0.3))

#adding Dense layer as an output
model.add(layers.Dense(1))

#for some tensorflow models we require to built model
model.build(input_shape=(None,128,128,3))

#compile model
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

#create check point to save best accuracy model
ckp_path = "trained_model\model"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,monitor="val_mae",mode="auto",save_best_only=True,save_weights_only=True)
#monitor: Monitor validation mae loss to save model
#mode: Use to save model when val_mae is maximum or minimum, it has 3 options "min","max","auto" for us we can select "min" or "auto"
#when val_mae reduce model will be saved
#save_best_only=False : It will save all model
#save_weights_only=True : Save only weights

#creating learning rate reducer to reduce lr when accuracy does not improve
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",mode="auto",cooldown=0,patience=5,verbose=1,min_lr=1e-6)
#factor: when it is reduce next lr will be 0.9 times of current
#patience=X
#reduce lr after X epoch when accuracy dose not improve
#verbose: show it after every epoch
#min_lr: minimum learning rate

#start training model
Epoch=100
Batch_size=32
#select batch size according to your graphics card
class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(X_train, Y_train, 32)
test_gen = DataGenerator(X_test, Y_test, 32)
#X_train,X_test,Y_train,Y_test
history = model.fit(train_gen,
                    validation_data=test_gen,
                    batch_size=Batch_size,
                    epochs=Epoch,
                    callbacks=[model_checkpoint,reduce_lr])
#Before training you can delete image_arr and lable_arr to increase RAM Memory

Epoch 1/100
558/558 [==============================] - 184s 241ms/step - loss: 1.8658 - mae: 1.8658 - val_loss: 2.1096 - val_mae: 2.1096 - lr: 0.0010
Epoch 2/100
558/558 [==============================] - 131s 236ms/step - loss: 0.7470 - mae: 0.7470 - val_loss: 0.6475 - val_mae: 0.6475 - lr: 0.0010
Epoch 3/100
558/558 [==============================] - 135s 241ms/step - loss: 0.5917 - mae: 0.5917 - val_loss: 0.3869 - val_mae: 0.3869 - lr: 0.0010
Epoch 4/100
558/558 [==============================] - 130s 233ms/step - loss: 0.5348 - mae: 0.5348 - val_loss: 0.4968 - val_mae: 0.4968 - lr: 0.0010
Epoch 5/100
558/558 [==============================] - 135s 242ms/step - loss: 0.5000 - mae: 0.5000 - val_loss: 0.6184 - val_mae: 0.6184 - lr: 0.0010
Epoch 6/100
558/558 [==============================] - 134s 240ms/step - loss: 0.4864 - mae: 0.4864 - val_loss: 0.2293 - val_mae: 0.2293 - lr: 0.0010
Epoch 7/100
558/558 [==============================] - 132s 237ms/step - loss: 0.4547 - mae: 0.4547 

In [4]:
# after the training is done load best model

model.load_weights(ckp_path)

# convert model to tensorflow lite model

converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

# save model
with open("model.tflite","wb") as f:
	f.write(tflite_model)

# if you want to see prediction result on test dataset
prediction_val=model.predict(X_test,batch_size=32)

# print first 10 values
print(prediction_val[:10])
# print first 10 values of Y_test
print(Y_test[:10])

# Save and run this python file
# Before that I will show you
# loss: 0.4074 - mae: 0.4074 - val_loss: 0.3797 - val_mae: 0.3797
# we have mae and val_mae:
# mae: Is on X_train
# val_mae: X_test
# If val_mae is reducing that means your model is improving.

INFO:tensorflow:Assets written to: C:\Users\UTKARS~1\AppData\Local\Temp\tmpmjc2gvzp\assets


INFO:tensorflow:Assets written to: C:\Users\UTKARS~1\AppData\Local\Temp\tmpmjc2gvzp\assets


99/99 [==============================] - 8s 54ms/step
[[19.98782  ]
 [29.83055  ]
 [30.02751  ]
 [22.021828 ]
 [22.02156  ]
 [ 4.947673 ]
 [ 4.949088 ]
 [27.859747 ]
 [ 2.9626722]
 [23.006702 ]]
[20. 30. 30. 22. 22.  5.  5. 28.  3. 23.]
